In [1]:
import sys
sys.path.insert(0, ".")

import singleton_helpers

In [2]:
from chia.wallet.puzzles import (singleton_top_layer, p2_delegated_puzzle_or_hidden_puzzle)
from cdv.test import Network, Wallet

network: Network = await Network.create()
await network.farm_block()

alice: Wallet = network.make_wallet("alice")
await network.farm_block(farmer=alice) # alice has 2000_000_000_000 mojos

print(f'alice: {alice.balance()}')
standard_txn_coin_wrapper = await alice.choose_coin(1_750_000_000_000)
standard_txn_coin = standard_txn_coin_wrapper.as_coin()
standard_txn_puzzle = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_pk(alice.pk())
standard_coin_message, coin_spends = singleton_helpers.get_create_singleton_coin_spends(
                    standard_txn_coin,
                    standard_txn_puzzle,
                    odd_amount = 1023, 
                    inner_puzzle = singleton_top_layer.adapt_inner_to_singleton(standard_txn_puzzle), 
                    keys_values = [("Key", "Value")]
              )
standard_txn_coin_spend = coin_spends[0]
launcher_coin_spend = coin_spends[1]
print("\nstarting coin spend:")
singleton_helpers.print_json(standard_txn_coin_spend.to_json_dict())
print("\nlauncher coin spend:")
singleton_helpers.print_json(launcher_coin_spend.to_json_dict())
print(f'\nlauncher id: {launcher_coin_spend.coin.name()}')
print(f'\nstandard_coin_message: {standard_coin_message.hex()}')

alice: 2000000000000

starting coin spend:
{
    "coin": {
        "amount": 1750000000000,
        "parent_coin_info": "0xe3b0c44298fc1c149afbf4c8996fb92400000000000000000000000000000001",
        "puzzle_hash": "0x4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3"
    },
    "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ff0bffff01ff02ffff03ffff09ff05ffff1dff0bffff1effff0bff0bffff02ff06ffff04ff02ffff04ff17ff8080808080808080ffff01ff02ff17ff2f80ffff01ff088080ff0180ffff01ff04ffff04ff04ffff04ff05ffff04ffff02ff06ffff04ff02ffff04ff17ff80808080ff80808080ffff02ff17ff2f808080ff0180ffff04ffff01ff32ff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff06ffff04ff02ffff04ff09ff80808080ffff02ff06ffff04ff02ffff04ff0dff8080808080ffff01ff0bffff0101ff058080ff0180ff018080ffff04ffff01b0a042c855d234578415254b7870b711fb25e8f85beaa4a66bd0673d394c761fa156406c2e3bb375d5b18766d2a12cc918ff018080",
    "solution": "0xff80ffff01ffff33ffa0eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da

In [3]:
from blspy import AugSchemeMPL, PrivateKey
from chia.consensus.default_constants import DEFAULT_CONSTANTS

synthetic_sk: PrivateKey = p2_delegated_puzzle_or_hidden_puzzle.calculate_synthetic_secret_key(
    alice.sk_,
    p2_delegated_puzzle_or_hidden_puzzle.DEFAULT_HIDDEN_PUZZLE_HASH
)

signature = AugSchemeMPL.sign(synthetic_sk,standard_coin_message)

In [4]:
from chia.types.spend_bundle import SpendBundle

spend_bundle = SpendBundle(
    coin_spends,
    signature
)
result = await network.push_tx(spend_bundle)
result

{'additions': [Coin(parent_coin_info=<bytes32: 12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba>, puzzle_hash=<bytes32: eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9>, amount=1023),
  Coin(parent_coin_info=<bytes32: 12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba>, puzzle_hash=<bytes32: 4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3>, amount=1749999998977),
  Coin(parent_coin_info=<bytes32: 6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570>, puzzle_hash=<bytes32: a1af73b7dc0f246ffb0bd41b4ac8f0253f4c0949bfdd070d52a3be037767baa3>, amount=1023)],
 'removals': [Coin(parent_coin_info=<bytes32: e3b0c44298fc1c149afbf4c8996fb92400000000000000000000000000000001>, puzzle_hash=<bytes32: 4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3>, amount=1750000000000),
  Coin(parent_coin_info=<bytes32: 12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba>, puzzle_hash=<bytes32: eff07522495060c06